In [5]:
import yfinance as yf
import pandas as pd
import os

# --- PARTIE 1 : DÉFINITION ET TÉLÉCHARGEMENT ---
tickers = ['AAPL', 'MSFT', 'GOOGL', '^GSPC'] # Actions + Indice S&P 500
start_date = '2019-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

print("Téléchargement des données en cours...")
data = yf.download(tickers, start=start_date, end=end_date)
data.index.name = 'Date' # Nommer l'index pour en faire une colonne dans le DataFrame final

C:\Users\USER\AppData\Local\Temp\ipykernel_10976\155162733.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[**********************50%                       ]  2 of 4 completed

Téléchargement des données en cours...


[*********************100%***********************]  4 of 4 completed


In [6]:
# --- PARTIE 2 : NETTOYAGE & RESTRUCTURATION ---
# Le prix de clôture est nommé 'Close' dans le MultiIndex de la structure des données
# Nous sélectionnons le niveau 'Close' et le convertissons au format 'long' (idéal pour Power BI)
try:
    df_closing_multiindex = data['Close']
    df_closing = df_closing_multiindex.stack(level=0).reset_index()
    df_closing.columns = ['Date', 'Ticker', 'Prix_Cloture']
    
except KeyError:
    print("Erreur: La colonne 'Close' est introuvable. Vérifiez les tickers.")
    exit()


In [7]:
# --- PARTIE 3 : FEATURE ENGINEERING ---
df_closing = df_closing.sort_values(by=['Ticker', 'Date'])

# 1. Rendement Quotidien
df_closing['Rendement'] = df_closing.groupby('Ticker')['Prix_Cloture'].pct_change()

# 2. Moyenne Mobile Simple (SMA) 50 et 200 jours
df_closing['SMA_50'] = df_closing.groupby('Ticker')['Prix_Cloture'].transform(
    lambda x: x.rolling(window=50).mean()
)
df_closing['SMA_200'] = df_closing.groupby('Ticker')['Prix_Cloture'].transform(
    lambda x: x.rolling(window=200).mean()
)

print("\nFeature Engineering terminé. Aperçu des données finales:")
print(df_closing.tail())


Feature Engineering terminé. Aperçu des données finales:
           Date Ticker  Prix_Cloture  Rendement       SMA_50      SMA_200
6815 2025-10-10  ^GSPC   6552.509766  -0.027112  6529.817373  6049.508789
6819 2025-10-13  ^GSPC   6654.720215   0.015599  6538.151582  6052.912041
6823 2025-10-14  ^GSPC   6644.310059  -0.001564  6544.438984  6055.933391
6827 2025-10-15  ^GSPC   6671.060059   0.004026  6551.876387  6059.100742
6831 2025-10-16  ^GSPC   6629.069824  -0.006294  6557.556582  6062.391892


In [8]:
# --- PARTIE 4 : ENREGISTREMENT SÉCURISÉ ---
# Enregistrement du fichier sur le Bureau pour une localisation facile
try:
    # Récupère le chemin du bureau de l'utilisateur
    chemin_bureau = os.path.join(os.path.expanduser('~'), 'Desktop', 'finance_features_final.csv')
    
    # Enregistre le fichier au chemin absolu
    df_closing.to_csv(chemin_bureau, index=False)
    
    print(f"\n✅ SUCCÈS : Le fichier est enregistré ici : {chemin_bureau}")

except Exception as e:
    print(f"Erreur lors de l'enregistrement: {e}")


✅ SUCCÈS : Le fichier est enregistré ici : C:\Users\USER\Desktop\finance_features_final.csv
